# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

talks
title	type	url_slug	venue	date	location	talk_url	description			
"Identity Driven Information Ecosystems: TikTok, Truckstops, and Barbershops"	Conference Presentation	ICA-2023	International Communication Association (ICA)	2023-05-27	"Toronto, ON, Canada"		"Hiaeshutter-Rice, D., Madrigal, G., Ploger, G., Carr, S., Carbone, M., <b> Battocchio, A.F.,</b> and Soroka, S. (2023). ""Identity Driven Information Ecosystems: TikTok, Truckstops, and Barbershops."" To be presented at the 73rd International Communication Association Conference, Toronto, Canada."			
Localizing social justice stories: Social media and local civic information infrastructure in six midwestern United States communities	Conference Presentation	AEJMC-2022-1	Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC)	2022-08-05	"Detroit, MI, USA"		"<b>Battocchio, A. F.,</b> Thorson, K., Hiaeshutter‑Rice, D., Smith, M., Chen, Y., Edgerly, S., Cotter, K., Denzin

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,,,,,,,,,,talks
title,type,url_slug,venue,date,location,talk_url,description,NaN,NaN,NaN
"Identity Driven Information Ecosystems: TikTok, Truckstops, and Barbershops",Conference Presentation,ICA-2023,International Communication Association (ICA),2023-05-27,"Toronto, ON, Canada",NaN,"Hiaeshutter-Rice, D., Madrigal, G., Ploger, G., Carr, S., Carbone, M., <b> Battocchio, A.F.,</b> and Soroka, S. (2023). ""Identity Driven Information Ecosystems: TikTok, Truckstops, and Barbershops."" To be presented at the 73rd International Communication Association Conference, Toronto, Canada.",NaN,NaN,NaN
Localizing social justice stories: Social media and local civic information infrastructure in six midwestern United States communities,Conference Presentation,AEJMC-2022-1,Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC),2022-08-05,"Detroit, MI, USA",NaN,"<b>Battocchio, A. F.,</b> Thorson, K., Hiaeshutter‑Rice, D., Smith, M., Chen, Y., Edgerly, S., Cotter, K., Denzin, K., Choung, H., Dong, C., Moldagaliyeva, M., Etheridge, C., Kohlmeier, S., and Werth, L. (2022). “Localizing social justice stories: Social media and local civic information infrastructure in six midwestern United States communities.” Presented in the Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC), Detroit, MI, August 5, 2022.",NaN,NaN,NaN
Discursive Opportunity Structure: The evolutions of racial justice discourses by local community organizations on Facebook,Conference Presentation,AEJMC-2022-2,Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC),2022-08-05,"Detroit, MI, USA",NaN,"Chen, Y., Choung, H., <b>Battocchio, A.F.</b>, Hiaeshutter-Rice, D., Smith, M., Draeger, M., Dong, C., and Xiao, A. (2022). “Discursive Opportunity Structure: The evolutions of racial justice discourses by local community organizations on Facebook.” Presented in the Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC), Detroit, MI, August 5, 2022.",NaN,NaN,NaN
Reaching the Rust Belt: Exploring identity and intergroup threat in 2020 election speeches,Conference Presentation,AEJMC-2022-3,Political Communication Division of the Association for Education in Journalism and Mass Communication(AEJMC),2022-08-03,"Detroit, MI, USA",NaN,"<b>Battocchio, A.F.</b> and Smith, M. (2022). ""Reaching the Rust Belt: Exploring identity and intergroup threat in 2020 election speeches."" Presented in the Political Communication Division of the Association for Education in Journalism and Mass Communication(AEJMC), Detroit, MI, August 3, 2022.",NaN,NaN,NaN
Hyperlocal affective polarization: Remixing rural understanding,Conference Presentation,AEJMC-2021-1,Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC),2021-08-06,Virtual,NaN,"<b>Battocchio, A.F. </b>(2021) “Hyperlocal affective polarization: Remixing rural understanding."" Presented in the Communication Theory and Method (CT&M) Division of the Association for Education in Journalism and Mass Communication (AEJMC), Virtual Conference, August 6, 2021.} <i>*awarded CT&M Top Student Paper: Second Place</I>",NaN,NaN,NaN
"“Do Good and Be ‘Liked'"": Corporate messaging on social media during COVID-19 and consumer responses",Conference Presentation,AEJMC-2021-2,Advertising Division of the Association for Education in Journalism and Mass Communication(AEJMC),2021-08-07,Virtual,NaN,"Yang, J., Bertellotti, E., <b>Battocchio, A.F.</b>, and Teran, C. (2021) “Do Good and Be ‘Liked'"": Corporate messaging on social media during COVID-19 and consumer responses."" Presented in the Advertising Division of the Association for Education in Journalism and Mass Communication(AEJMC), Virtual Conference, August 7, 2021.",NaN,NaN,NaN
A systematic method of cat

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

AttributeError: 'Series' object has no attribute 'date'

These files are in the talks directory, one directory below where we're working from.

In [ ]:
!ls ../_talks

2012-03-01-talk-1.md	  2014-02-01-talk-2.md
2013-03-01-tutorial-1.md  2014-03-01-talk-3.md


In [ ]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
